# Modeling Data and saving in the GOLD layer

## Creation of dim_company

In [0]:
%sql
CREATE OR REPLACE TABLE gold.marketing.dim_company AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY Company) AS company_id,  -- Unique ID for each company
    Company AS company_name,
    Customer_Segment
FROM silver.marketing.marketing_data;

## Creation of dim_campaign

In [0]:
%sql
CREATE OR REPLACE TABLE gold.marketing.dim_campaign AS
SELECT 
    Campaign_ID AS campaign_key,  -- Original Campaign ID as primary identifier
    ROW_NUMBER() OVER (ORDER BY Campaign_ID) AS campaign_id,  -- Surrogate ID
    Campaign_Type,
    Target_Audience,
    Duration,
    Campaign_Efficiency
FROM silver.marketing.marketing_data;

## Creation of dim_channel

In [0]:
%sql
CREATE OR REPLACE TABLE gold.marketing.dim_channel AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY Channel_Used) AS channel_id,  -- Unique ID for each channel
    Channel_Used AS channel_name,
    Language
FROM silver.marketing.marketing_data;

## Creation of dim_location

In [0]:
%sql
CREATE OR REPLACE TABLE gold.marketing.dim_location AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY Location) AS location_id,  -- Unique ID for each location
    Location AS location_name,
    Age_Group
FROM silver.marketing.marketing_data;

## Creation of the fact_campaign_performance

In [0]:
%sql
CREATE OR REPLACE TABLE gold.marketing.fact_campaign_performance AS
SELECT 
    cam.campaign_id,
    com.company_id,
    ch.channel_id,
    loc.location_id,
    SUM(s.Clicks) AS Total_Clicks,
    SUM(s.Impressions) AS Total_Impressions,
    AVG(s.Conversion_Rate) AS Avg_Conversion_Rate,
    SUM(s.Acquisition_Cost) AS Total_Acquisition_Cost,
    AVG(s.ROI) AS Avg_ROI,
    MAX(s.Campaign_Efficiency) AS Campaign_Efficiency  -- Assume best efficiency rating
    
FROM silver.marketing.marketing_data s
JOIN gold.marketing.dim_company com ON s.Company = com.company_name
JOIN gold.marketing.dim_campaign cam ON s.Campaign_ID = cam.campaign_key
JOIN gold.marketing.dim_channel ch ON s.Channel_Used = ch.channel_name
JOIN gold.marketing.dim_location loc ON s.Location = loc.location_name
GROUP BY 
    cam.campaign_id, com.company_id, ch.channel_id, loc.location_id;
